# Build a Model

Dan thinks a next great step: 
* basis of building a model.
* i.e. try to predict star rating (1-5) based off review text... in class different options for taking in review text and converting it to something we can reason about.
* For each review, create something that is a 1-hot encoding of that review. i.e. for all words used in corpus.

In [ ]:
# # some possible useful functions courtesy of DL exercise chat: 

# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer()
# from sklearn.model_selection import train_test_splitfrom sklearn.linear_model import LogisticRegressionfrom sklearn import metrics


# x_train, x_test, y_train, y_test = train_test_split(reviews, ratings)

# lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')





In [1]:
# !pip3 install spacy

In [2]:
# !python -m spacy download en_core_web_sm

In [3]:
import gzip
import json
import spacy
import pandas as pd

In [4]:
# Read in grocery and gourmet food data

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [5]:
df = getDF('Grocery_and_Gourmet_Food_5.json.gz')

In [6]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"11 19, 2014",A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,1416355200,NaN,NaN,NaN
1,5.0,True,"10 13, 2016",A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,1476316800,NaN,NaN,NaN
2,5.0,True,"11 21, 2015",A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,1448064000,NaN,NaN,NaN
3,5.0,True,"08 12, 2015",A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,1439337600,NaN,NaN,NaN
4,5.0,True,"05 28, 2015",A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,1432771200,NaN,NaN,NaN


In [8]:
# Interested in columns overall and reviewText

df_preproc = cleaner(df)
df_preproc.head(3) = df[["overall", "reviewText"]]

In [9]:
# Clean reviewText

# import spacy 
print("Row count is:", len(df_all.index))
print("Number of reviewText null is: ", df_all["reviewText"].isnull().values.sum())
print("Number of overall null is: ", df_all["overall"].isnull().values.sum())

df_all = df_all.dropna()
print("Clean row count is:", len(df_all.index))


Row count is: 1143860
Number of reviewText null is:  390
Number of overall null is:  0
Clean row count is: 1143470


In [18]:
# Use subset for now - write code to work with just 1000?
n_data_samples = 1000
df_data = df_all.head(n_data_samples)

In [14]:
df_data.head()

,overall,reviewText
0,5.0,No adverse comment.
1,5.0,Gift for college student.
2,5.0,"If you like strong tea, this is for you. It mi..."
3,5.0,Love the tea. The flavor is way better than th...
4,5.0,I have searched everywhere until I browsed Ama...


In [19]:
# Use spacy to remove stop words and craete list of words for each reviewText row?
# nlp = spacy.load('en_core_web_sm')
# spacy pipeline capability. Rather than mapping directly. 

from sklearn.feature_extraction import text
stop = text.ENGLISH_STOP_WORDS

# Should all these transforms be in my read function? Probably... so I can load data efficiently. 

# split string into list, all lower case, remove punctuation, remove stop words
df_data['text_split'] = df_data['reviewText'].str.replace('[^\w\s]','').str.lower().str.split()
df_data['text_no_stop'] = df_data['text_split'].apply(lambda x: [item for item in x if item not in stop])



/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [20]:
df_data.head(5)

,overall,reviewText,text_split,text_no_stop
0,5.0,No adverse comment.,"[no, adverse, comment]","[adverse, comment]"
1,5.0,Gift for college student.,"[gift, for, college, student]","[gift, college, student]"
2,5.0,"If you like strong tea, this is for you. It mi...","[if, you, like, strong, tea, this, is, for, yo...","[like, strong, tea, little, strong]"
3,5.0,Love the tea. The flavor is way better than th...,"[love, the, tea, the, flavor, is, way, better,...","[love, tea, flavor, way, better, regular, lipt..."
4,5.0,I have searched everywhere until I browsed Ama...,"[i, have, searched, everywhere, until, i, brow...","[searched, browsed, amazon, tea, lipton, selli..."


In [22]:
nlp = spacy.load("en_core_web_sm")  # load an existing English template


ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'list'>

In [19]:
df_data.to_csv("df_data_clean.csv")  

In [39]:
# Do better
# FOlllowing https://prrao87.github.io/blog/spacy/nlp/performance/2020/05/02/spacy-multiprocess.html

In [35]:
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])
nlp.add_pipe('sentencizer')

In [27]:
from sklearn.feature_extraction import text

stopwords = text.ENGLISH_STOP_WORDS

In [32]:
import re

def cleaner(df):
    "Extract relevant text from DataFrame using a regex"
    # Regex pattern for only alphanumeric, hyphenated text with 3 or more chars
    pattern = re.compile(r"[A-Za-z0-9\-]{3,50}")
    df['clean'] = df['reviewText'].str.findall(pattern).str.join(' ')
    return df

In [33]:
df_preproc = cleaner(df_data)
df_preproc.head(3)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,overall,reviewText,text_split,text_no_stop,clean
0,5.0,No adverse comment.,"[no, adverse, comment]","[adverse, comment]",adverse comment
1,5.0,Gift for college student.,"[gift, for, college, student]","[gift, college, student]",Gift for college student
2,5.0,"If you like strong tea, this is for you. It mi...","[if, you, like, strong, tea, this, is, for, yo...","[like, strong, tea, little, strong]",you like strong tea this for you might even li...


In [36]:
def lemmatize_pipe(doc):
    lemma_list = [str(tok.lemma_).lower() for tok in doc
                  if tok.is_alpha and tok.text.lower() not in stopwords] 
    return lemma_list

def preprocess_pipe(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append(lemmatize_pipe(doc))
    return preproc_pipe

In [39]:
from joblib import Parallel, delayed

def chunker(iterable, total_length, chunksize):
    return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))

def flatten(list_of_lists):
    "Flatten a list of lists to a combined list"
    return [item for sublist in list_of_lists for item in sublist]

def process_chunk(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append(lemmatize_pipe(doc))
    return preproc_pipe

def preprocess_parallel(texts, chunksize=100):
    executor = Parallel(n_jobs=7, backend='multiprocessing', prefer="processes")
    do = delayed(process_chunk)
    tasks = (do(chunk) for chunk in chunker(texts, len(df_preproc), chunksize=chunksize))
    result = executor(tasks)
    return flatten(result)

In [40]:
df_preproc['preproc_parallel'] = preprocess_parallel(df_preproc['clean'], chunksize=1000)
df_preproc.head(3)

/home/ec2-user/.local/lib/python3.7/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,overall,reviewText,text_split,text_no_stop,clean,preproc_pipe,preproc_parallel
0,5.0,No adverse comment.,"[no, adverse, comment]","[adverse, comment]",adverse comment,"[adverse, comment]","[adverse, comment]"
1,5.0,Gift for college student.,"[gift, for, college, student]","[gift, college, student]",Gift for college student,"[gift, college, student]","[gift, college, student]"
2,5.0,"If you like strong tea, this is for you. It mi...","[if, you, like, strong, tea, this, is, for, yo...","[like, strong, tea, little, strong]",you like strong tea this for you might even li...,"[like, strong, tea, little, strong]","[like, strong, tea, little, strong]"


In [42]:
df_use = df_preproc[["overall", "preproc_parallel"]]

In [43]:
df_use.head(3)

,overall,preproc_parallel
0,5.0,"[adverse, comment]"
1,5.0,"[gift, college, student]"
2,5.0,"[like, strong, tea, little, strong]"


In [45]:
# create corpus and corresponding one-hot dictionary
# something like:
all_words = ' '.join([word for word in df_use['preproc_parallel']])


TypeError: sequence item 0: expected str instance, list found

In [74]:
all_words = [item for sublist in df_use['preproc_parallel'] for item in sublist]
corpus = set(all_words)
word_to_ix = {word: i for i, word in enumerate(corpus)}

In [75]:
word_to_ix

{'moister': 0,
 'cigars': 1,
 'linked': 2,
 'colored': 3,
 'blue': 4,
 'ridiculously': 5,
 'acidic': 6,
 'subscribe': 7,
 'stamped': 8,
 'wasatch': 9,
 'embarrassed': 10,
 'awful': 11,
 'party': 12,
 'coronary': 13,
 'regulation': 14,
 'smoothies': 15,
 'handful': 16,
 'impact': 17,
 'acids': 18,
 'fundraiser': 19,
 'guy': 20,
 'conditions': 21,
 'singing': 22,
 'wanted': 23,
 'poultry': 24,
 'sell': 25,
 'aim': 26,
 'sign': 27,
 'mocha': 28,
 'stove': 29,
 'experienced': 30,
 'staled': 31,
 'looks': 32,
 'months': 33,
 'static': 34,
 'wild': 35,
 'radiation': 36,
 'ache': 37,
 'life': 38,
 'marshmallows': 39,
 'alcohols': 40,
 'carbs': 41,
 'stars': 42,
 'lime': 43,
 'inverted': 44,
 'words': 45,
 'standby': 46,
 'delicious': 47,
 'flowing': 48,
 'pollen': 49,
 'years': 50,
 'issue': 51,
 'pull': 52,
 'basil': 53,
 'imho': 54,
 'introduced': 55,
 'shelves': 56,
 'blew': 57,
 'recipe': 58,
 'economical': 59,
 'rarely': 60,
 'fancy': 61,
 'enclosed': 62,
 'granular': 63,
 'occasion': 64

In [76]:
# 10 most common words? 
from collections import Counter

Counter = Counter(all_words)
  
# most_common() produces k frequently encountered
# input values and their respective counts.
most_occur = Counter.most_common(10)
  
print(most_occur)

[('great', 244), ('flavor', 239), ('good', 236), ('like', 234), ('gum', 159), ('love', 152), ('tea', 145), ('use', 142), ('taste', 141), ('just', 138)]
